In [1]:
%matplotlib widget
%config InlineBackend.figure_format = 'svg'

In [2]:
import addict
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from importlib import reload
from tqdm.notebook import tqdm

import celeri
celeri = reload(celeri)

# Plotting the global model is much much faster with tex fonts turned off
plt.rcParams['text.usetex'] = False

In [3]:
command_file_name = "./data/western_north_america/basic_command.json"
command, segment, block, meshes, station, mogi, sar = celeri.read_data(command_file_name)
station = celeri.process_station(station, command)
segment = celeri.process_segment(segment, command, meshes)
sar = celeri.process_sar(sar, command)
closure, block = celeri.assign_block_labels(segment, station, block, mogi, sar)
# celeri.plot_block_labels(segment, block, station, closure)

In [4]:
assembly = addict.Dict()
operators = addict.Dict()
assembly = celeri.merge_geodetic_data(assembly, station, sar)
assembly, operators.block_motion_constraints = celeri.block_constraints(assembly, block, command)
assembly, operators.slip_rate_constraints = celeri.slip_rate_constraints(assembly, segment, block, command)
operators.okada_segment_station = celeri.get_segment_station_operator_okada(segment, station, command)
#celeri.plot_segment_displacements(segment, station, command, segment_idx=0, strike_slip=1, dip_slip=0, tensile_slip=0, lon_min=235, lon_max=255, lat_min=30, lat_max=50, quiver_scale=1e-1)
operators.slip_rate_segment_block = celeri.get_fault_slip_rate_partials(segment, block)
assembly.index.sz_elastic = operators.okada_segment_station.shape # Not sure this is correct
assembly.index.sz_slip = operators.slip_rate_segment_block.shape # Not sure this is correct
operators.block_rotation = celeri.get_block_rotation_operator(station, block)
assembly.index.sz_rotation = operators.block_rotation.shape # Not sure this is correct
assembly = celeri.station_row_keep(assembly) # Not sure this is correct
operators.block_strain_rate, strain_rate_block_idx = celeri.get_strain_rate_centroid_operator(block, station, segment) # This function need work


Calculating Okada partials for segments:   0%|          | 0/837 [00:00<?, ?it/s]

In [30]:
def get_mogi_operator(mogi, station, command):
    """
    # Calculates partial derivatives for Mogi sources
    # Uses Equation 7.14 of "Earthquake and Volcano Deformation" by P. Segall
    """
    mogi_depth = celeri.KM2M * mogi.depth
    poissons_ratio = command.material_mu / (2 * (command.material_lambda + command.material_mu))
    mogi_operator = np.zeros((3 * len(station), len(mogi)))

    for i in range(len(mogi)):
        # Calculate great circle distance and azimuth between source and stations
        # source_to_obs_distance = gcdist(Mogi.lon(i), Mogi.lat(i), Station.lon, Station.lat)
        # source_to_obs_azimuth = sphereazimuth(Mogi.lon(i), Mogi.lat(i), Station.lon, Station.lat)
    
        # Calculate partials relating displacement (rate) to unit volume change (rate)
        # Segall Equation 7.14
        u_up = (1 - poissons_ratio) / np.pi * mogi_depth[i] / ((source_to_obs_distance ** 2.0 + mogi_depth[i] ** 2) ** 1.5)
        u_radial = (1 - poissons_ratio) / np.pi * source_to_obs_distance / ((source_to_obs_distance ** 2 + mogi_depth[i] ** 2.0) ** 1.5)
    
        # Convert radial displacement to east and north components
        # TODO: Do I have to convert for geologic strike to Cartesian angle???
        u_east = u_radial * np.sin(source_to_obs_azimuth)
        u_north = u_radial * np.cos(source_to_obs_azimuth)
    
        #Insert components into partials matrix
        mogi_operator[0::3, i] = u_east
        mogi_operator[1::3, i] = u_north
        mogi_operator[2::3, i] = u_up
    return mogi_operator

# def mogi_forward(mogi, station, command):
#     """
#     Calculate displacements from a single Mogi source using
#     equation 7.14 from "Earthquake and Volcano Deformation" by Paul Segall
#     """
    # Calculate great circle distance and azimuth between source and stations

# i = 0
# mogi_depth = celeri.KM2M * mogi.depth
# poissons_ratio = command.material_mu / (2 * (command.material_lambda + command.material_mu))

# source_to_obs_distance = gcdist(mogi.lon[i], mogi.lat[i], station.lon, station.lat)
# source_to_obs_azimuth = sphereazimuth(mogi.lon[i], mogi.lat[i], Station.lon, Station.lat)

# # Calculate partials relating displacement (rate) to unit volume change (rate)
# # Segall Equation 7.14
# u_up = (1 - poissons_ratio) / np.pi * mogi_depth[i] / ((source_to_obs_distance ** 2.0 + mogi_depth[i] ** 2) ** 1.5)
# u_radial = (1 - poissons_ratio) / np.pi * source_to_obs_distance / ((source_to_obs_distance ** 2 + mogi_depth[i] ** 2.0) ** 1.5)

# # Convert radial displacement to east and north components
# # TODO: Do I have to convert for geologic strike to Cartesian angle???
# u_east = u_radial * np.sin(source_to_obs_azimuth)
# u_north = u_radial * np.cos(source_to_obs_azimuth)
    

# % Make some figures for testing

# % Segall Figure 7.5
# %figure
# %plot(dist(:, 1)./Mogi.dep(i), ur./max(uz));
# %hold on
# %plot(dist(:, 1)./Mogi.dep(i), uz./max(uz));
# %xlabel('Radial distance (r/depth)')
# %ylabel('Normalized displacement')
# %
# %figure('position', [100 100 1000 200]);
# %subplot(1, 4, 1)
# %scatter(Station.lon, Station.lat, 25, uz, 'filled');
# %title('Vertical displacement')
# %subplot(1, 4, 2)
# %scatter(Station.lon, Station.lat, 25, ur, 'filled');
# %title('Radial displacement')
# %subplot(1, 4, 3)
# %scatter(Station.lon, Station.lat, 25, ue, 'filled');
# %title('East displacement')
# %subplot(1, 4, 4)
# %scatter(Station.lon, Station.lat, 25, un, 'filled');
# %title('North displacement')

mogi = addict.Dict()
mogi.lon = np.array([20])
mogi.lat = np.array([45])
mogi.depth = np.array([5])
sta = addict.Dict()
lon_mat, lat_mat = np.meshgrid(np.linspace(19, 21, 20), np.linspace(44, 46, 20))
sta.lon = lon_mat.flatten()
sta.lat = lat_mat.flatten()

mogi_depth = celeri.KM2M * mogi.depth
poissons_ratio = command.material_mu / (2 * (command.material_lambda + command.material_mu))

u_east = np.zeros_like(sta.lon)
u_north = np.zeros_like(sta.lon)
u_up = np.zeros_like(sta.lon)
for i in range(sta.lon.size):
    source_to_obs_forward_azimuth, source_to_obs_backward_azimuth, source_to_obs_distance = celeri.GEOID.inv(mogi.lon[0], mogi.lat[0], sta.lon[i], sta.lat[i])
    # Calculate partials relating displacement (rate) to unit volume change (rate)
    # Segall Equation 7.14
    u_up[i] = (1 - poissons_ratio) / np.pi * mogi.depth[0] / ((source_to_obs_distance ** 2.0 + mogi_depth[0] ** 2) ** 1.5)
    u_radial = (1 - poissons_ratio) / np.pi * source_to_obs_distance / ((source_to_obs_distance ** 2 + mogi_depth[0] ** 2.0) ** 1.5)

    # Convert radial displacement to east and north components
    # TODO: Do I have to convert for geologic strike to Cartesian angle???
    u_east[i] = u_radial * np.sin(np.deg2rad(source_to_obs_forward_azimuth))
    u_north[i] = u_radial * np.cos(np.deg2rad(source_to_obs_forward_azimuth))

plt.figure()
plt.quiver(sta.lon, sta.lat, u_east, u_north)
plt.gca().set_aspect("equal")
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
mogi.lon[i]

20

In [21]:
sta.lon.size

400

In [14]:
segment_forward_azimuth

90.54441342829426